## 马尔可夫决策过程
### 基本概念
是强化学习很重要的观念，含义是说当前状态受到前面状态的影响，一般来说，强化学习中的**环境**就是一个马尔可夫决策过程
### 马尔可夫过程
#### 随机过程
指概率论中的"动态学"部分，概率论的研究对象是静态的随机现象，但是随机过程研究的是随时间推演的随机现象（比如天气变化等等）
#### 马尔可夫性质
当且仅当某一时刻的状态取决于上一时刻的状态时，一个随机过程被称为具有马尔可夫性质
#### 马尔可夫过程
指具有马尔可夫性质的随机过程，也称为马尔可夫链，通常用元组{S,P}组成，S指有限数量的**状态**集合，P是**状态转移矩阵**
#### 马尔可夫奖励过程
在马尔可夫过程的基础上加入**奖励函数**和**折扣因子**，可以得到马尔可夫奖励过程。一个马尔可夫奖励过程由{S,P,r,γ}组成
- S是有限状态的集合
- P是状态转移矩阵
- r是奖励函数，某个状态的奖励r(s)指转移到该状态时可以获得的奖励期望
- γ时折扣因子，取值为[0,1],引入折扣因子是因为远期利益具有一定的不确定性，有时更希望能够尽快获得一些奖励，因此需要对远期利益大一些折扣。γ趋近1则更关注长期积累奖励；趋近于0则更考虑短期奖励
#### 马尔可夫回报
当前状态的回报依赖于当前的奖励值和后面一直到终止状态的奖励衰减和

![image-2.png](https://github.com/115545863/RL-startLevel/blob/main/picture/1.png)

计算的时候会倒着计算，因为我计算当前步，我会用到后续所有的还没计算出来的部分，但是我倒着计算，就没有不知道的值了，一个一个都给算出来了

![image.png](https://github.com/115545863/RL-startLevel/blob/main/picture/2.png)


In [32]:
# 初始化
import numpy as np

# 计算回报
p = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]

p = np.array(p)

rewards = [-1,-2,-2,10,1,0]
gamma = 0.5

In [33]:
# 计算回报
def Grade(start_index, chain, gamma):
    # 记录累计回报
    G = 0
    G1 = 0
    # 记录次数
    count = 0
    # 记录长度
    length = len(chain)-start_index-1
    for i in reversed(range(start_index, len(chain))):
        # 每次迭代的时候都要把前一个状态乘γ，这样其实得到的结果就是γ的n-1次方，合理的
        # 算法一（迭代倒叙公式，但是结果跟2是一样的）
        G = gamma*G + rewards[chain[i]-1]
        print(G)
        # 算法2（严格遵守图片公式）
        G1 += gamma**(length-count)*rewards[chain[i]-1]
        print(G1)
        count+=1
    return G

# 验证
# 一个状态序列,s1-s2-s3-s6
chain = [1, 2, 3, 6]
start_index = 0
G = Grade(start_index, chain, gamma)
# print("根据本序列计算得到回报为：%s。" % G)


0.0
0.0
-2.0
-0.5
-3.0
-1.5
-2.5
-2.5


#### 价值函数
与回报相关，上述内容求的是特定一条路线情况下的值，而价值函数则是该状态所在的所有路线的值的期望（平均值）

![image-3.png](picture\3.png)

上式就是贝尔曼方程，当然简单理解其实就是一个求期望嘛。根据这个公式，能够推导出下面的公式：

![image-3.png](picture\4.png)

将所有的状态价值表示为列向量，同时将奖励函数也携程列向量，可以得到下面的值

![image-3.png](picture\5.png)

为什么左右都是V(s)呢，因为从理解上来讲，这个只是说我求当前状态价值，需要下一个状态的价值分数，但是下一个状态也有可能还是这个状态，就是他从自己跳到自己状态了，所以这样计算是合理的
从含义上说，左边就是当前状态的函数，右边是下一个状态，还是这个意思

时间复杂度是O(n**3), n是状态数，所以只适用于很小的马尔可夫奖励过程，求解大规模的还是得使用动态规划、蒙特卡洛和时序差分等等

In [34]:
# 贝尔曼方程
def bellman(p,rewards,gamma,states_num):
    # 转换为列向量，1是指定第二维的长度是1（列长是1），-1是指根据长度自动划分第一维大小，确保总数不变（比如有6个数，列数为1，那么行数就得是6）
    rewards = np.array(rewards).reshape(-1,1)
    # 价值函数
    # np.eye(states_num, states_num) 构建单位矩阵
    # np.linalg.inv(np.eye(states_num,states_num)-gamma*p) 求逆矩阵
    # np.dot(np.linalg.inv(np.eye(states_num,states_num)-gamma*p),rewards) 矩阵相乘
    value = np.dot(np.linalg.inv(np.eye(states_num,states_num)-gamma*p),rewards)

    return value
V = bellman(p,rewards,gamma, len(rewards))
print("MRP中每个状态价值分别为\n", V)

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


### 马尔可夫决策过程
前面提到的过程都是自发改变的随机过程，但如果有一个外界的刺激来共同改变这个随机过程，就称为马尔可夫决策过程，这个外界刺激被称为智能体的动作。
马尔可夫决策过程由元组{S,A,P,r,γ}组成
- S是有限状态的集合
- A是动作的集合
- P(s'|s,a)是状态转移函数，表示在状态s执行动作a之后到达状态s'的概率
- r(s,a)是奖励函数，此奖励可以同时取决于状态s和动作a，在奖励函数只取决于状态s时，则退化为r(s)
- γ时折扣因子
这个智能体/外界刺激的概念可以理解为，小船在海上航行最后抵达目的地，这是一个马尔可夫奖励过程；但是如果由一个水手来掌舵，能够更好更高效的到达目的地，这就是马尔可夫决策过程，水手就是那个智能体/外界刺激

![image-3.png](picture\6.png)


#### 策略
智能体的策略使用Π来表示，策略Π(a|s)=P(A = a| S = s)是一个函数，表示在输入状态s的情况下采取动作a的概率。
- **确定性策略**：每个状态只输出一个确定性的动作，即只有该动作的概率1，其他概率都是0
- **随机性策略**：在每个状态时输出的是关于动作的概率分布，然后根据该分布进行采样可以得到一个动作
#### 相关函数
- **状态价值函数**：表示MDP中的基于策略的状态价值函数

![image-3.png](picture\7.png)

- **动作价值函数**：额外定义一个动作价值函数，下面公式1表示的是在策略Π的情况下，受状态和动作影响的价值函数；公式2表示是状态价值函数和动作价值函数之间的关系，在使用策略Π中，状态s的价值等于在该状态下基于策略Π采取的所有动作的概率和对应价值相乘求和的结果；公式3指的是在使用策略Π时，状态s下采取的动作a的价值等于即时奖励加上经过衰减后的所有可能的下一个状态的状态转移概率与对应的价值的乘积

![image-3.png](picture\8.png)

#### 贝尔曼期望方程
如下图所示

![image-3.png](picture\9.png)

下图是一个马尔可夫决策的例子，绿色圆圈表示一个状态，一共有五个状态，黑色实线表示可以采取的动作，黄色小圆圈表示动作。某个动作只有和他相关联的状态可以采取，别的不可以。数字表示采取该动作获得的奖励，虚线箭头表示该动作后可以转移到的状态，箭头旁边的分数表示概率，即采取该动作的概率

![image-3.png](picture\10.png)


In [35]:
# 马尔可夫决策过程例子
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}


# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2

In [36]:
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = bellman(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]
